In [9]:
import requests
import re
import string
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

### Вакансии

Ищем ссылки на вакансии с ключевым словом IT, геолокацией -- Россия (по умолчанию отдается Москва), пока страница с указанным номером существует

In [19]:
main_url = "https://www.superjob.ru"
search_url = main_url + "/vacancy/search/"

vacancies_urls = []
i = 0
while True:
    url_params = {'keywords': 'IT', 'geo[c][0]': '1', 'page': i}
    vacancies_html = requests.get(search_url, params=url_params).text
    soup = BeautifulSoup(vacancies_html, 'html.parser')
    not_found_div = soup.find('div', class_='_3OLv-')
    if not_found_div:
        break
    vacancies_div = soup.find('div', class_='_1ID8B')
    vacancies_div = vacancies_div.find('div', class_='_3VcZr')
    vacancies_links = vacancies_div.find_all('a', class_='_2JivQ')
    urls = list(map(lambda link: main_url + link.attrs['href'], vacancies_links))
    vacancies_urls += urls
    if (i % 10) == 0:
        print("page:", i, "collected:", len(vacancies_urls))
    i += 1
print("page_count:", i)

page: 0 collected: 30
page: 10 collected: 330
page: 20 collected: 630
page: 30 collected: 929
page_count: 31


In [20]:
print(len(vacancies_urls))
vacancies_urls[:10]

929


['https://www.superjob.ru/vakansii/menedzher-it-proektov-31595397.html',
 'https://www.superjob.ru/vakansii/it-specialist-31654573.html',
 'https://www.superjob.ru/vakansii/it-menedzher-po-upravleniyu-relizami-31604972.html',
 'https://www.superjob.ru/vakansii/veduschij-it-analitik-31604896.html',
 'https://www.superjob.ru/vakansii/starshij-it-menedzher-po-upravleniyu-relizami-31604850.html',
 'https://www.superjob.ru/vakansii/analitik-v-otdel-avtomatizacii-kreditno-depozitnyh-produktov-31604865.html',
 'https://www.superjob.ru/vakansii/glavnyj-it-analitik-31604884.html',
 'https://www.superjob.ru/vakansii/it-specialist-31595180.html',
 'https://www.superjob.ru/vakansii/it-arhitektor-31604739.html',
 'https://www.superjob.ru/vakansii/menedzher-it-riskov-31604699.html']

Удалим дубликаты

In [21]:
uniq = np.array(vacancies_urls)
vacancies_urls = np.unique(uniq)
print(len(vacancies_urls))
vacancies_urls[:10]

848


array(['https://www.superjob.ru/vakansii/administrative-specialist-31675371.html',
       'https://www.superjob.ru/vakansii/administrativnyj-assistent-31673685.html',
       'https://www.superjob.ru/vakansii/administrativnyj-direktor-31663053.html',
       'https://www.superjob.ru/vakansii/administrator-31611832.html',
       'https://www.superjob.ru/vakansii/administrator-31682464.html',
       'https://www.superjob.ru/vakansii/administrator-baz-dannyh-31601600.html',
       'https://www.superjob.ru/vakansii/administrator-kassir-31684759.html',
       'https://www.superjob.ru/vakansii/administrator-proektov-31614556.html',
       'https://www.superjob.ru/vakansii/administrator-seti-31670845.html',
       'https://www.superjob.ru/vakansii/administrator-sistemy-monitoringa-31477488.html'],
      dtype='<U159')

Соберу те вакансии, у которых указана зарплата

In [24]:
def collect_vacancy(soup):
    result = {}
    div = soup.find('div', class_='iJCa5')
    result['name'] = div.find('h1').text
    
    text_spans = soup.find_all('span', class_='_1h3Zg _1K0Mf ADNB4')
    if len(text_spans) > 0:
        result['address'] = text_spans[0].text
        if len(text_spans) > 1:
            result['employment'] = text_spans[1].text

    salary = soup.find('span', class_='_1h3Zg _2Wp8I _1fqdH ADNB4 _1BoTZ').\
        find_all('span')
    salary = [str(s) for s in salary]
    result['salary'] = re.sub('[<span/\xa0>\s]', '', ''.join(salary))
    if not result['salary']:
#         print("no salary determined")
        return

    exp_span = soup.find('span', class_='_1h3Zg _1K0Mf ADNB4 _37Ti9')
    if exp_span:
        result['experience'] = exp_span.text
    
    category = soup.find('span', class_='_1h3Zg z15vz _1xK5K').\
        find_all('span')
    category = set([s.find('a').text for s in category])
    category.remove('Вакансии')
    category.remove('Работа в Москве')
    result['categories'] = ', '.join(category)

    text_spans = soup.find_all('span', class_='_1h3Zg sI2a4 ADNB4')
    if len(text_spans) > 0:
        result['duties'] = re.sub('\n', '', text_spans[0].text)
        if len(text_spans) > 1:
            result['requirements'] = re.sub('\n', '', text_spans[1].text)
            if len(text_spans) > 2:
                result['conditions'] = re.sub('\n', '', text_spans[2].text)
    return result

result = []
for url_id, url in enumerate(vacancies_urls):
    #print(url_id, url)
    vacancy_html = requests.get(url).text
    soup = BeautifulSoup(vacancy_html, 'html.parser')
    soup = soup.find('div', class_='_3Qutk')
    vacancy = collect_vacancy(soup)
    if vacancy:
        vacancy['url'] = url
#         for k, v in vacancy.items():
#             print(k, ":", v)
        result.append(vacancy)
    if (url_id % 10) == 0:
        print("url:", url_id, "processed")

url: 0 processed
url: 10 processed
url: 20 processed
url: 30 processed
url: 40 processed
url: 50 processed
url: 60 processed
url: 70 processed
url: 80 processed
url: 90 processed
url: 100 processed
url: 110 processed
url: 120 processed
url: 130 processed
url: 140 processed
url: 150 processed
url: 160 processed
url: 170 processed
url: 180 processed
url: 190 processed
url: 200 processed
url: 210 processed
url: 220 processed
url: 230 processed
url: 240 processed
url: 250 processed
url: 260 processed
url: 270 processed
url: 280 processed
url: 290 processed
url: 300 processed
url: 310 processed
url: 320 processed
url: 330 processed
url: 340 processed
url: 350 processed
url: 360 processed
url: 370 processed
url: 380 processed
url: 390 processed
url: 400 processed
url: 410 processed
url: 420 processed
url: 430 processed
url: 440 processed
url: 450 processed
url: 460 processed
url: 470 processed
url: 480 processed
url: 490 processed
url: 500 processed
url: 510 processed
url: 520 processed
url:

In [25]:
len(result)

530

In [26]:
df = pd.DataFrame(result)
print(df.shape)
df.head()

(530, 10)


,address,categories,conditions,duties,employment,experience,name,requirements,salary,url
0,"Крылатское, Москва, Осенний бульвар, 12к1","Администратор-кассир, Продажи",Плавающий гибкий график работы 2/2;Оформление ...,Lamoda – одна из наиболее успешных и динамично...,NaN,сменный график работы,Администратор-кассир,Доброжелательность и коммуникабельность - в пр...,25000—40000₽,https://www.superjob.ru/vakansii/administrator...
1,"Москва, Походный пр-д, д. 3 к. 2","Администратор сети, IT, Интернет, связь, телеком",Оформление в соответствии с ТК РФ с первого ра...,Администрирование текущей системы мониторинга ...,NaN,сменный график работы,Администратор системы мониторинга (Zabbix),Образование незаконченное высшее/ высшее.Поним...,40000—50000₽,https://www.superjob.ru/vakansii/administrator...
2,"Уфа, ул. Карла Маркса, д. 46","Строительство, проектирование, недвижимость, А...","Добрый день, уважаемый кандидат!Если Вы сейчас...",• сопровождать сделки с недвижимостью;• анализ...,NaN,полная занятость,Агент по недвижимости,• желание много зарабатывать и готовность вкла...,50000₽,https://www.superjob.ru/vakansii/agent-po-nedv...
3,"Воронеж, ул. Революции 1905 года, д. 31г","Агент по недвижимости, Продажи","Центр недвижимости ""36-я Авеню"" в связи с расш...",Что делать:успешное прохождение обучения;презе...,NaN,полная занятость,Агент по продаже жилой недвижимости (Центр нед...,От Вас:нацеленность на достижение материальног...,35000—100000₽,https://www.superjob.ru/vakansii/agent-po-prod...
4,"Московская обл., Ногинский р-н, тер Ногинск-Те...","IT, Интернет, связь, телеком, Аналитик",Стабильная крупная производственная компания с...,"Принимать участие в сборе, проверке и анализе ...",полная занятость,опыт работы от 1 года,Аналитик,"Высшее техническое образование,Опыт работы в а...",43000₽,https://www.superjob.ru/vakansii/analitik-3163...


In [27]:
with open('data/vacancies.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv)

\t(sdf)